In [4]:
import os
import json
import requests
from web3 import Web3
from dotenv import load_dotenv

load_dotenv()

rpc_url = "https://poly-rpc.gateway.pokt.network/"
web3 = Web3(Web3.HTTPProvider(rpc_url))
block_number = web3.eth.blockNumber
print(f"Connected: {web3.isConnected()}, block number: {block_number}")


Connected: True, block number: 34397091


In [5]:
contract_address = Web3.toChecksumAddress("0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA")
abi_filename = (
    "../resources/71937/Devcon_Offset_Pool_0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA.json"
)

def load_abi(filename):
    with open(filename, "r") as f:
        abi = json.load(f)
    return abi

abi = load_abi(abi_filename)
pooling_contract = web3.eth.contract(address=contract_address, abi=abi)

In [6]:
contributor_addresses = pooling_contract.functions.getContributorsAddresses().call()
contributed_nct = pooling_contract.functions.totalCarbonPooled().call()

print("Contributor count: ", len(contributor_addresses))
print("Amount of NCT contributed:", web3.fromWei(contributed_nct, "ether"))

Contributor count:  168
Amount of NCT contributed: 369.953925268267454384


In [7]:
api_key = os.environ.get("POLYGONSCAN_API_KEY")
if not api_key:
    print("error: please add your polygonscan api key to the .env")
deployment_block_number = 33870323
current_block_number = web3.eth.blockNumber
base_url = "https://api.polygonscan.com/api?"
params = (f"module=account"
            f"&action=txlist"
            f"&address={contract_address}"
            f"&startblock={deployment_block_number}"
            f"&endblock={current_block_number}"
            f"&sort=asc"
            f"&apikey={api_key}")
print("Getting...", base_url + params)
response = requests.get(base_url + params)
print(response.status_code)
assert response.status_code == 200, f"Oops, request unsuccessful, {response.reason}"


Getting... https://api.polygonscan.com/api?module=account&action=txlist&address=0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA&startblock=33870323&endblock=34397093&sort=asc&apikey=221BQBBF24Q657NBSGTSEHW4IXZ6EDU812
200


In [8]:
import pandas as pd
response_content = json.loads(response.content)
transaction_list = response_content['result']
print("Transaction count", len(transaction_list))

erc20_names = {
    "0xD838290e877E0188a4A44700463419ED96c16107": "NCT",
    "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174": "USDC",
    "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063": "DAI",
    "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619": "WETH",
    "0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270": "WMATIC",
    }

erc20_count = 0
matic_count = 0
matic_sent = 0
txn_error_count = 0
data = []
for txn in transaction_list:
    if int(txn["isError"]):
        txn_error_count += 1
        continue
    if txn['functionName'].startswith("participateWithToken"):
        erc20_count += 1
        for erc20_address, erc20_name in erc20_names.items():
            # amount_nct = 
            if erc20_address == Web3.toChecksumAddress(txn["input"][34:74]):
                if erc20_name == "USDC":
                    decimals = 6
                else:
                    decimals = 18
                data.append(
                    {"token": erc20_name, 
                     "amount": int(txn['value'])/10**decimals,
                     "blocknumber": int(txn['blockNumber']),
                     "timestamp": int(txn["timeStamp"]),
                     "nonce": int(txn["nonce"]),
                     "from": Web3.toChecksumAddress(txn["from"]),
                     "amount NCT": None
                    }
                    )
    elif txn['functionName'].startswith("participateWithMatic"):
        data.append(
            {"token": "MATIC", 
             "amount": int(txn['value'])/10**18,
             "blocknumber": int(txn['blockNumber']),
             "timestamp": int(txn["timeStamp"]),
             "nonce": int(txn["nonce"]),
             "from": Web3.toChecksumAddress(txn["from"]),
             "amount NCT": None
            }
        )
        matic_sent += int(txn['value'])
    else:
        print(txn['functionName'])
print("txn_error", txn_error_count)
df = pd.DataFrame(data)

Transaction count 176
atInversebrah(int248 a, uint48[] b, uint32 c, bytes20[] d, bytes30[] e)
txn_error 0


In [9]:
import plotly.express as px
import random

fig1 = px.pie(df.token, values=df.token.value_counts().values, names=df.token.value_counts().index, title = 'Climate Friendly Devcon Token Usage', width=800, height=800)
fig1.update_traces(title_font = dict(size=24), textinfo='label+value', textposition='inside')
fig1.show()

In [14]:
df.timestamp = pd.to_datetime(df['timestamp'],unit='s')
# fig2 = px.histogram(df, x=df.timestamp, y=df.amount, title="Climate Friendly Devcon Participation")
fig2.show()